# **Parkinson's Disease Detection**

**Objective:** To develop a precise and efficient system for early Parkinson's disease detection utilizing multiple deep learning models. This project aims to harness the capabilities of various deep learning architectures to analyze diverse biomarkers and clinical data, thereby enhancing diagnostic accuracy and enabling proactive healthcare interventions for improved patient outcomes.

**Importing necessary libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Loading dataset and defining train and test generators**

In [ ]:
image_directory = "/content/drive/MyDrive/Parkinson datasets/Parkinson's image dataset"

datagen=ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2
)

train_generator=datagen.flow_from_directory(
    image_directory,
    target_size=(224,224),
    batch_size=64,
    class_mode='binary',
    subset='training'
)

test_generator=datagen.flow_from_directory(
    image_directory,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 665 images belonging to 2 classes.
Found 166 images belonging to 2 classes.


# **CNN model**

In [ ]:
cnn_model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1000,activation='relu'),
    tf.keras.layers.Dense(2,activation='softmax')

])



cnn_model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


cnn_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train','Test'],loc="upper left")
plt.show()


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train','Test'],loc="upper left")
plt.show()

Epoch 1/10
11/11 [==============================] - 156s 13s/step - loss: 0.6492 - accuracy: 0.7338 - val_loss: 0.5928 - val_accuracy: 0.7349
Epoch 2/10
11/11 [==============================] - 8s 748ms/step - loss: 0.6069 - accuracy: 0.7338 - val_loss: 0.5583 - val_accuracy: 0.7349
Epoch 3/10
11/11 [==============================] - 6s 498ms/step - loss: 0.5943 - accuracy: 0.7338 - val_loss: 0.5523 - val_accuracy: 0.7349
Epoch 4/10
11/11 [==============================] - 5s 472ms/step - loss: 0.5853 - accuracy: 0.7338 - val_loss: 0.5448 - val_accuracy: 0.7349
Epoch 5/10
11/11 [==============================] - 5s 420ms/step - loss: 0.5796 - accuracy: 0.7338 - val_loss: 0.5406 - val_accuracy: 0.7349
Epoch 6/10
11/11 [==============================] - 6s 495ms/step - loss: 0.5705 - accuracy: 0.7338 - val_loss: 0.5360 - val_accuracy: 0.7349
Epoch 7/10
11/11 [==============================] - 5s 426ms/step - loss: 0.5624 - accuracy: 0.7338 - val_loss: 0.5257 - val_accuracy: 0.7349
Epoch 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score

# Evaluate the model on the test set
cnn_model.evaluate(test_generator)

# Get predictions for the test set
cnn_predictions = cnn_model.predict(test_generator)

# Convert softmax predictions to class labels
cnn_predictions_classes = np.argmax(cnn_predictions, axis=1)

# Get true class labels
true_labels = test_generator.classes

# Evaluate precision
cnn_precision = precision_score(true_labels, cnn_predictions_classes, average='weighted')
print("CNN Precision:", cnn_precision)

# Evaluate AUC-ROC
cnn_auc_roc = roc_auc_score(true_labels, cnn_predictions[:, 1], average='weighted')
print("CNN AUC-ROC:", cnn_auc_roc)

6/6 [==============================] - 1s 137ms/step
CNN Precision: 0.5401364494121063
CNN AUC-ROC: 0.5031669150521609


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **SVM model**

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score

# Assuming train_generator and test_generator are instances of ImageDataGenerator

# Get the number of samples in the training and testing sets
num_train_samples = len(train_generator.filenames)
num_test_samples = len(test_generator.filenames)

# Flatten the images for SVM
train_data_svm = cnn_model.predict(train_generator)
train_data_svm = train_data_svm.reshape(num_train_samples, -1)

test_data_svm = cnn_model.predict(test_generator)
test_data_svm = test_data_svm.reshape(num_test_samples, -1)

# SVM model
svm_model = svm.SVC()
svm_model.fit(train_data_svm, train_generator.classes)

# Predictions
svm_predictions = svm_model.predict(test_data_svm)

# Evaluate accuracy
svm_accuracy = accuracy_score(test_generator.classes, svm_predictions)
print("SVM Accuracy:", svm_accuracy)

# Evaluate precision
svm_precision = precision_score(test_generator.classes, svm_predictions, zero_division=1)
print("SVM Precision:", svm_precision)

# Evaluate AUC-ROC
svm_auc_roc = roc_auc_score(test_generator.classes, svm_predictions)
print("SVM AUC-ROC:", svm_auc_roc)

6/6 [==============================] - 1s 141ms/step
SVM Accuracy: 0.7349397590361446
SVM Precision: 1.0
SVM AUC-ROC: 0.5


# **Logistic Regression model**

In [ ]:
from sklearn.linear_model import LogisticRegression

# Flatten the images for Logistic Regression
train_data_lr = cnn_model.predict(train_generator)
train_data_lr = train_data_lr.reshape(num_train_samples, -1)

test_data_lr = cnn_model.predict(test_generator)
test_data_lr = test_data_lr.reshape(num_test_samples, -1)

# Logistic Regression model
lr_model = LogisticRegression()
lr_model.fit(train_data_lr, train_generator.classes)

# Predictions
lr_predictions = lr_model.predict(test_data_lr)

# Evaluate accuracy
lr_accuracy = accuracy_score(test_generator.classes, lr_predictions)
print("Logistic Regression Accuracy:", lr_accuracy)

# Evaluate precision
lr_precision = precision_score(test_generator.classes, lr_predictions, zero_division=1)
print("Logistic Regression Precision:", lr_precision)

# Evaluate AUC-ROC
lr_auc_roc = roc_auc_score(test_generator.classes, lr_model.predict_proba(test_data_lr)[:, 1])
print("Logistic Regression AUC-ROC:", lr_auc_roc)

6/6 [==============================] - 1s 131ms/step
Logistic Regression Accuracy: 0.7349397590361446
Logistic Regression Precision: 1.0
Logistic Regression AUC-ROC: 0.569672131147541


# **RNN model**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, TimeDistributed, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Assuming train_generator and test_generator are instances of ImageDataGenerator

# Reshape data into sequences
sequence_length = 10  # Choose an appropriate length for your sequences
train_sequence_generator = TimeseriesGenerator(
    train_generator[0][0],  # Input data (images) from train_generator
    train_generator[0][1],  # Target data (labels) from train_generator
    length=sequence_length,
    batch_size=64
)

test_sequence_generator = TimeseriesGenerator(
    test_generator[0][0],  # Input data (images) from test_generator
    test_generator[0][1],  # Target data (labels) from test_generator
    length=sequence_length,
    batch_size=32
)

# Build the RNN model
rnn_model = Sequential([
    TimeDistributed(Flatten(input_shape=(224, 224, 3))),  # Flatten each image in the sequence
    SimpleRNN(64, activation='relu', input_shape=(sequence_length, 224*224*3)),
    Dense(2, activation='softmax')
])

# Compile the model
rnn_model.compile(optimizer=SGD(lr=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
rnn_model.fit(train_sequence_generator, epochs=10, validation_data=test_sequence_generator)

# Evaluate the model
rnn_loss, rnn_accuracy = rnn_model.evaluate(test_sequence_generator)
print("RNN Accuracy:", rnn_accuracy)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.5270 - accuracy: 0.8148 - val_loss: 4.0324 - val_accuracy: 0.8636
Epoch 2/10
1/1 [==============================] - 1s 931ms/step - loss: 2.3998 - accuracy: 0.8333 - val_loss: 0.8213 - val_accuracy: 0.5909
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: 0.5988 - accuracy: 0.7407 - val_loss: 0.8636 - val_accuracy: 0.8636
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 0.4945 - accuracy: 0.8333 - val_loss: 0.5459 - val_accuracy: 0.8636
Epoch 5/10
1/1 [==============================] - 1s 940ms/step - loss: 0.4963 - accuracy: 0.9259 - val_loss: 1.2748 - val_accuracy: 0.8636
Epoch 6/10
1/1 [==============================] - 1s 835ms/step - loss: 0.5723 - accuracy: 0.8333 - val_loss: 0.7356 - val_accuracy: 0.8636
Epoch 7/10
1/1 [==============================] - 1s 949ms/step - loss: 0.3438 - accuracy: 0.8889 - val_loss: 0.7176 - val_accuracy: 0.8636
Epoch 8/10
1/1 [=============

In [ ]:
from sklearn.metrics import precision_score, roc_auc_score

# Assuming test_sequence_generator is an instance of TimeseriesGenerator

# Get true class labels
true_labels = test_sequence_generator.targets

# Get predictions for the test set
rnn_predictions = rnn_model.predict(test_sequence_generator)

# Convert softmax predictions to class labels
rnn_predictions_classes = np.argmax(rnn_predictions, axis=1)

# Evaluate precision
rnn_precision = precision_score(true_labels, rnn_predictions_classes, average='weighted')
print("RNN Precision:", rnn_precision)

# Evaluate AUC-ROC
rnn_auc_roc = roc_auc_score(true_labels, rnn_predictions[:, 1], average='weighted')
print("RNN AUC-ROC:", rnn_auc_roc)

1/1 [==============================] - 0s 457ms/step


ValueError: Found input variables with inconsistent numbers of samples: [32, 22]